## 특성공학

- 기존 특성을 사용해서 새로운 특성을 뽑아내는 작업을 '특성 공학'이라고 함

### Perch종의 무게 예측

In [12]:
import pandas as pd

df = pd.read_csv('data/Fish.csv')
perch_df = df.loc[df['Species'] == 'Perch']
perch_df.head()

# 각 특성을 주고 그에 맞는 
# 무게 찾는 것을 목표로 함

,Species,Weight,Length1,Length2,Length3,Height,Width
72,Perch,5.9,7.5,8.4,8.8,2.1120,1.4080
73,Perch,32.0,12.5,13.7,14.7,3.5280,1.9992
74,Perch,40.0,13.8,15.0,16.0,3.8240,2.4320
75,Perch,51.5,15.0,16.2,17.2,4.5924,2.6316
76,Perch,70.0,15.7,17.4,18.5,4.5880,2.9415


In [45]:
perch_full = perch_df[['Length2', 'Height', 'Width']] # 학습시킬 특성만 뽑기
perch_weight = perch_df[['Weight']] # 목표(타겟)인 무게만 뽑아둠

In [42]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
train_test_split(perch_full, perch_weight)

# 가지고 있는 데이터 셋을 학습과 테스트 세트로 나눔

In [23]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=False)
# 내가 특성을 주면 그 특성으로 만들 수 있는 여러가지 조합을 보여줌
# 결과값에 1을 포함시킬지 말지(include_bias)
# True를 쓰는 경우는 b(절편)를 직접 다룰 때 사용  ** 솔직히 뭔말인지 모르겠..
# linear regression이 자동으로 절편을 처리하므로 보통 False로 사용

poly.fit([[3, 5]])
# 여기서 fit은 역할이 없지만, 꼭 써줘야 함

poly.transform([[3, 5]])
# 3과 5를 가지고 만들어 낼 수 있는 모든 경우의 수를 만들어줌

array([[ 3.,  5.,  9., 15., 25.]])

In [32]:
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
# 원본 데이터를 새롭게 poly라는 것으로 만들어줌


train_poly.shape
# 우리가 가지고 있는 생선은 42마리,
# 특성을 9가지 만듦

(42, 9)

In [35]:
poly.get_feature_names_out()
# poly객체의 메서드를 사용
# 다항변환을 통해 생성된 특성들의 이름 반환
# (fit()을 통해 학습한 특성 변환 규칙을 기반으로 특성 이름 만들어줌)

array(['Length2', 'Height', 'Width', 'Length2^2', 'Length2 Height',
       'Length2 Width', 'Height^2', 'Height Width', 'Width^2'],
      dtype=object)

In [38]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_poly, train_target)
lr.score(train_poly, train_target)

0.9933618280482341

In [40]:
test_poly = poly.transform(test_input)

In [41]:
lr.score(test_poly, test_target)

0.9653489323249087

## 규제
- 특성의 스케일이 정규화되지 않으면 곱해지는 계수 값도 차이가 남
- 일반적으로 선형 회귀 모델에 규제를 적용할 때
- 계수 값의 크기가 서로 많이 다르면 공정하게 제어되지 않음
- 규제를 적용하기 전에 먼저 정규화 필요

In [79]:
poly = PolynomialFeatures(degree=5, include_bias=False)
# degree=5 는 5차 다항식을 의미, x의 5제곱까지의 특성 만듦
# 높은 degree는 복잡한 관계를 반영할 수 있지만 과적합의 위험이 있음

poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
train_poly.shape

# 생선은 42마리, 특성은 55가지 생성

(42, 55)

In [50]:
lr.fit(train_poly, train_target)

LinearRegression()

In [53]:
print(lr.score(train_poly, train_target))

print(lr.score(test_poly, test_target))
# 학습에 너무 최적화된 나머지, 실제로 test 값을 평가할 때는 방해가 됨

0.9999999999999987
-16576.850746301014


In [60]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
# 데이터를 표준화하는 도구 (평균 0, 표준편차 1로 변환)
ss.fit(train_poly)
# 55개의 데이터를 분석하여 평균과 표준편차 계산

train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)
# transform은 학습된 규칙(평균, 표준편차)을 데이터에 적용하여 표준화된 데이터 생성
# 이때 학습 데이터와 테스트 데이터 둘 다 동일한 규칙을 사용해 표준화해야함!

In [62]:
train_scaled[:5]

array([[ 0.60305087,  0.72090749,  0.834245  ,  0.4853013 ,  0.55117081,
         0.62161256,  0.60269682,  0.67502082,  0.73257335,  0.35638644,
         0.40083015,  0.45229453,  0.43832288,  0.49165681,  0.53653668,
         0.46909218,  0.52415166,  0.57088372,  0.60863119,  0.22827811,
         0.26060534,  0.3004527 ,  0.28900527,  0.33050648,  0.36648338,
         0.31339405,  0.35649187,  0.39404773,  0.42550428,  0.33370841,
         0.37831815,  0.41743848,  0.45046532,  0.47706717,  0.10878347,
         0.13305177,  0.16469976,  0.15498865,  0.1880001 ,  0.2172826 ,
         0.17445382,  0.20880711,  0.23938452,  0.26571272,  0.19131584,
         0.22696786,  0.25884259,  0.28642592,  0.30939836,  0.20546368,
         0.24234373,  0.27549545,  0.30436334,  0.32858817,  0.3479864 ],
       [ 1.12325449,  1.03843768,  1.48013874,  1.10767315,  1.05292034,
         1.35820291,  0.98981597,  1.28332934,  1.57965208,  1.06406326,
         1.01988896,  1.27207671,  0.96965583,  1.

In [67]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(train_scaled, train_target)

# ridge 규제는 기본적으로 Alpha = 1.0
# 최적의 Alpha값을 찾으려면 반복을 여러번 돌려야함

Ridge()

In [72]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.1) # 가장 최적화된 알파값임
ridge.fit(train_scaled, train_target)

# ridge 규제는 기본적으로 Alpha = 1.0
# 최적의 Alpha값을 찾으려면 반복을 여러번 돌려야함

Ridge(alpha=0.1)

In [73]:
print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))

0.9889211997751844
0.9853577716448811


In [74]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(train_scaled, train_target)

print(lasso.score(train_scaled, train_target))
print(lasso.score(test_scaled, test_target))

0.9868137994912051
0.988779265462844


In [75]:
lasso.coef_
# 실제로 라쏘모델이 사용한 특성은 15개뿐

array([ 0.        ,  0.        ,  0.        , 29.71177309, 33.48980267,
       63.3538546 ,  0.        ,  0.        , 22.09135902, 17.69658322,
       56.61667517,  0.        , 41.26057662,  0.        ,  0.        ,
       19.4757107 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       39.05342576,  0.        ,  0.        ,  0.        , 28.60123515,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        , -0.        ])